# Imports

In [111]:
# Pandas is an open source data analysis and manipulation tool
import pandas as pd
from pandas import json_normalize

# os gives access to the operating system
import os
# The datetime module supplies classes for manipulating dates and times.
from datetime import datetime
import datetime
# This module provides various time-related functions.
import time

# Natural language toolkit
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# library to create visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.offsetbox import AnchoredText


# open source library for automating downloading of reports from Google Trends
from pytrends.request import TrendReq

# library to get html of website (wikipedia)
import requests
# json to use wikipedia return
import json

import nltk
#nltk.download('punkt')
#nltk.download('stopwords')





# Load data

In [112]:
# Folderpath to CSV's
CSV_folderPath="C:/Users/Jan/Documents/Python_Projects/Bachelorthesis/Bachelorthesis/CSV_Backlog"

# Initialize list<csv>
Combi_csvs = []

# Change directory to CSV_folderPath
os.chdir(CSV_folderPath)

# iterate over all files
for file in os.listdir():
    # that are csvs...
    if file.endswith(".csv"):
        print(file)
        # read csv to dataframe
        df =pd.read_csv(file, index_col=None, header=0,encoding = "utf-8")
        # all df append to combi...
        Combi_csvs.append(df)
All_articles = pd.concat(Combi_csvs, axis=0, ignore_index=True)

Bild_01012022_13_20_40.csv
Bild_01022022_13_41_10.csv
Bild_01032022_08_07_43.csv
Bild_01042022_08_50_35.csv
Bild_01052022_12_11_05 (1).csv
Bild_01052022_12_11_05 (10).csv
Bild_01052022_12_11_05 (100).csv
Bild_01052022_12_11_05 (1000).csv
Bild_01052022_12_11_05 (1001).csv
Bild_01052022_12_11_05 (1002).csv
Bild_01052022_12_11_05 (1003).csv
Bild_01052022_12_11_05 (1004).csv
Bild_01052022_12_11_05 (1005).csv
Bild_01052022_12_11_05 (1006).csv
Bild_01052022_12_11_05 (1007).csv
Bild_01052022_12_11_05 (1008).csv
Bild_01052022_12_11_05 (1009).csv
Bild_01052022_12_11_05 (101).csv
Bild_01052022_12_11_05 (1010).csv
Bild_01052022_12_11_05 (1011).csv
Bild_01052022_12_11_05 (1012).csv
Bild_01052022_12_11_05 (1013).csv
Bild_01052022_12_11_05 (1014).csv
Bild_01052022_12_11_05 (1015).csv
Bild_01052022_12_11_05 (1016).csv
Bild_01052022_12_11_05 (1017).csv
Bild_01052022_12_11_05 (1018).csv
Bild_01052022_12_11_05 (1019).csv
Bild_01052022_12_11_05 (102).csv
Bild_01052022_12_11_05 (1020).csv
Bild_01052022_12

## Merg names of different news

In [113]:
All_articles.loc[All_articles['News_page'] == "Bild"]

,Titel,Overline,Date_Info,Kategorie,URL,detailed_informations,bild_plus,Zugriff_Datum,News_page,Breadcrumb,author
0,Haben Sie heute auch mehr Lust auf Sex?,„Horny Hangover“,2022-01-01 10:59:40,Erotik,/bild-plus/unterhaltung/erotik/erotik/mehr-lus...,\r\n Wenn Sie a...,True,1012022,Bild,NaN,NaN
1,Präses Aust: Wir sind wie auf Schiff in schwer...,Versammlung eines Ehrbaren Kaufmanns,2021-12-31 18:12:43,Hamburg,/regional/hamburg/hamburg-aktuell/versammlung-...,\r\n 140 Kaufl...,False,1012022,Bild,NaN,NaN
2,Das Köln-Jahr zwischen Abstiegs-Angst und Att...,Menschen & Momente 2021,2021-12-31 12:58:42,Bundesliga,/sport/fussball/fussball/1-fc-koeln-menschen-m...,\r\n WAS FÜR EI...,False,1012022,Bild,NaN,NaN
3,"Bekomme ich Geld zurück, wenn ich nicht tanzen...",Teure Silvester-Tickets,2021-12-31 10:41:38,NaN,/sparfochs/2021/sparfochs/teure-eintrittskarte...,\r\n Wegen Coro...,False,1012022,Bild,NaN,NaN
4,Die schönste Love Story des Jahres,J.Lo und Ben,2021-12-31 08:35:55,Leute,/unterhaltung/leute/leute/jennifer-lopez-und-b...,\r\n Amor traf ...,False,1012022,Bild,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
60194,Auf diese Ziele fliegt Rhein-Main,Wohin in den Urlaub?,2021-12-31 08:45:23,Frankfurt,/regional/frankfurt/frankfurt-aktuell/wo-reise...,\r\n Schlangen ...,False,31122021,Bild,NaN,NaN
60195,Dashcam filmt Glatteis-Chaos in Oregon,Immer mehr Unfälle,2021-12-30,News Ausland,https://www.bild.de/video/clip/video/immer-meh...,\r\n Im US-Bund...,False,31122021,Bild,NaN,NaN
60196,Was für Halstenberg auf dem Spiel steht,Neuer Vertrag & WM,2021-12-30 18:19:38,Bundesliga,/sport/fussball/fussball/vertrag-und-wm-was-fu...,\r\n Kein Spiel...,False,31122021,Bild,NaN,NaN
60197,Epstein-Komplizin Ghislaine Maxwell schuldig,Geschworene haben Urteil gefällt,2021-12-30 15:02:28,News Ausland,/news/ausland/news-ausland/geschworene-haben-u...,\r\n Die Geschw...,False,31122021,Bild,NaN,NaN


## Data Cleaning

In [114]:
# lower categories
All_articles["Kategorie"] = All_articles["Kategorie"].str.lower()

# cast date info
All_articles["Date_Info"] = All_articles["Date_Info"].str.strip()

All_articles["Date_Info"] = pd.to_datetime(All_articles["Date_Info"], format='%d%m%Y_%H:%M', errors="coerce").fillna(pd.to_datetime(All_articles["Date_Info"], format="%Y-%m-%d %H:%M:%S", errors="coerce").fillna(pd.to_datetime(All_articles["Date_Info"], format="%d%m%Y_%H:%M", errors="coerce")))

# Drop duplicates
All_articles= All_articles.drop_duplicates(subset="Titel")


## Only use data after 01.10.2022 because there was a lag of data before


In [115]:
#greater than the start date and smaller than the end date
start_date = pd.to_datetime("2022 1 10")
end_date = pd.to_datetime('today')

All_articles = All_articles.loc[All_articles["Date_Info"].between(start_date, end_date)]

## Ammount of articles


In [116]:
# Spiegel articles
ammount_of_Spiegel_articles = len(All_articles.loc[All_articles['News_page'] == "Spiegel"])
print(f"Spiegel articles: {ammount_of_Spiegel_articles}")

# Sueddeutsche articles
ammount_of_Sueddeutsche_articles = len(All_articles.loc[All_articles['News_page'] == "Sueddeutsche"])
print(f"Sueddeutsche articles: {ammount_of_Sueddeutsche_articles}")

# Bild articles
ammount_of_Bild_articles = len(All_articles.loc[All_articles['News_page'] == "Bild"])
print(f"Bild articles: {ammount_of_Bild_articles}")

# All articles
ammount_of_Combined_articles = len(All_articles)
print(f"Overall: {ammount_of_Combined_articles}")

Spiegel articles: 11695
Sueddeutsche articles: 68021
Bild articles: 39835
Overall: 119551


## Merge Titel and Overline
Titel und Overline werden nun zusammengefügt um die Thematik des Artikels besser herauszuarbeiten

In [117]:
All_articles["Titel_and_Overline"] = All_articles["Titel"] +" "+All_articles["Overline"]
All_articles["Titel_and_Overline"] = All_articles["Titel_and_Overline"].fillna(All_articles["Titel"])
All_articles["Titel_and_Overline"] = All_articles["Titel_and_Overline"].astype(str)

## Segmentation
Segmentierung von Titel_and_Overline

1. Step
Create new column with Titel_and_Overline as list

In [118]:
All_articles["Tokens"] = All_articles.apply(lambda row: nltk.word_tokenize(row['Titel_and_Overline']), axis=1)

2. Step
Explode dataframe on Title_and_Overline_list

In [119]:
All_articles_workingCopy = All_articles.copy()
All_articles = All_articles_workingCopy.explode("Tokens")

## Remove stopwords

In [120]:
All_articles = All_articles[~All_articles['Tokens'].isin(stopwords.words("german"))]
# more stopwords
# continues extension
uninteresting_titles= ["Der", "Die", "Das","Was","Warum","Er","Sie", "Es", "Ich", "Du", "Mit","Wie","Ein","So","Wir","Nach","Am","An","B","Bei","Beim","Bin","Bis","Co","Da","A","C",":","»","«", "," ,"\'\'","``", "_","-" ".","?","–", "-",".","!"]
All_articles=All_articles[~All_articles.Tokens.isin(uninteresting_titles)]

## Remove words that start with lower

In [121]:
All_articles = All_articles[All_articles["Tokens"].str.istitle()]

In [122]:
All_articles["Tokens"].value_counts()[0:20]

Fußball         6006
Berlin          4418
Gesundheit      3995
Kriminalität    3878
Ukraine         3028
München         2499
Hamburg         2244
Krieg           2142
Unfälle         2120
Konflikte       2106
Polizei         1989
Mann            1714
Bayern          1691
Euro            1603
Verkehr         1567
Russland        1497
Auto            1491
Stuttgart       1442
Hannover        1388
Wetter          1329
Name: Tokens, dtype: int64

In [123]:
# ammount of words
len(All_articles["Tokens"])

540950

## Stemming
Wird nicht verwendet da es keine guten Ergebnisse erzielt

In [ ]:
#stemmer = nltk.SnowballStemmer('german')
#All_articles["Tokens_stemmed"] = All_articles["Tokens"].apply(stemmer.stem)



In [13]:
All_articles.loc[All_articles['Tokens'] == "Fußball"]



,Titel,Overline,Date_Info,Kategorie,URL,detailed_informations,bild_plus,Zugriff_Datum,News_page,Breadcrumb,author,Titel_and_Overline,Tokens
1341,"Fußball ohne Zuschauer, Weihnachtsmärkte dicht!",Knallhart-Regeln in BaWü,2021-11-30 17:30:26,stuttgart,/regional/stuttgart/stuttgart-aktuell/kretschm...,\r\n Ministerpr...,False,1122021,Bild,NaN,NaN,"Fußball ohne Zuschauer, Weihnachtsmärkte dicht...",Fußball
2014,Sylvester Stallone plant TV-Serie mit Boateng,Es ist nicht Fußball,2022-03-02 10:36:57,leute,/unterhaltung/leute/leute/sylvester-stallone-p...,\r\n Vom Fußbal...,False,2032022,Bild,NaN,NaN,Sylvester Stallone plant TV-Serie mit Boateng...,Fußball
6661,Der Fußball sollte sagen: Scheiß drauf!,Die Lage der 2. Liga,2022-03-05 21:22:31,bundesliga,/sport/fussball/fussball/die-lage-der-2-liga-d...,\r\n Schalkes T...,False,6032022,Bild,NaN,NaN,Der Fußball sollte sagen: Scheiß drauf! Die La...,Fußball
7368,Verkehrschaos rund um Leipziger Arenen erwartet,Fußball und Handball,2022-04-07 12:04:17,leipzig,/regional/leipzig/leipzig-news/leipzig-verkehr...,\r\n Rund um di...,False,7042022,Bild,NaN,NaN,Verkehrschaos rund um Leipziger Arenen erwarte...,Fußball
7747,"„Wir spielen Fußball mit Herz""",Matchwinner Baumgartl,2021-12-06 14:26:55,bundesliga,/sport/fussball/fussball/matchwinner-baumgartl...,\r\n Tor und Vo...,False,7122021,Bild,NaN,NaN,"„Wir spielen Fußball mit Herz"" Matchwinner Bau...",Fußball
...,...,...,...,...,...,...,...,...,...,...,...,...,...
273068,Fußball-Club-WM im Februar 2022 in den Emiraten,Fußball,2021-11-29 09:45:00,sport,https://www.sueddeutsche.de/sport/fussball-fus...,NaN,NaN,29112021,Sueddeutsche,dpa,NaN,Fußball-Club-WM im Februar 2022 in den Emirate...,Fußball
273070,"Hitzlsperger warnt vor Geisterspielen: ""Dramat...",Fußball,2021-11-29 09:42:00,sport,https://www.sueddeutsche.de/sport/fussball-hit...,NaN,NaN,29112021,Sueddeutsche,dpa,NaN,"Hitzlsperger warnt vor Geisterspielen: ""Dramat...",Fußball
273075,Neuer Sportvorstand: Mislintat setzt VfB-Führu...,Fußball,2021-11-29 09:35:00,sport,https://www.sueddeutsche.de/sport/fussball-neu...,NaN,NaN,29112021,Sueddeutsche,dpa,NaN,Neuer Sportvorstand: Mislintat setzt VfB-Führu...,Fußball
273078,Hertha trennt sich von Trainer Dardai - Korkut...,Fußball,2021-11-29 09:28:00,sport,https://www.sueddeutsche.de/sport/fussball-her...,NaN,NaN,29112021,Sueddeutsche,dpa,NaN,Hertha trennt sich von Trainer Dardai - Korkut...,Fußball


## Create KeyWord list

In [124]:
keyWords=All_articles.groupby("Tokens").size()
keyWords = pd.DataFrame({"KeyWord":keyWords.index,"Occurence":keyWords.values})

## Articles that occure more than n times

In [125]:
occurence = [0,10,25,50,75,100,125,150,200,300,350,400,450,500]
for n in occurence:
    keyWords_by_N = keyWords.loc[keyWords["Occurence"] >= n]
    print("n:"+str(n) +" = "+str(len(keyWords_by_N)))

n:0 = 89807
n:10 = 7060
n:25 = 2981
n:50 = 1485
n:75 = 984
n:100 = 729
n:125 = 557
n:150 = 450
n:200 = 297
n:300 = 181
n:350 = 148
n:400 = 125
n:450 = 107
n:500 = 95


In [126]:
def saveCSV(dataframe, filename):
    dataframe.to_csv("C:/Users/Jan/Documents/Python_Projects/Bachelorthesis/Bachelorthesis/Analysis/DataFrames/"+ filename +".csv",index=False)

In [127]:
saveCSV(keyWords,"keyWords")